# Set Environment

In [1]:
# libraries
import numpy as np
import pandas as pd
from collections import Counter
pd.options.display.max_columns = None
pd.options.display.max_rows = 300

In [2]:
#Define variables
Y_lab = 'good' # col name of Y (PD)
rs = 2020 #random state
col_sep = ':'

In [3]:
# Import Train and Test Data.
old_data = pd.read_csv('processed_data/train_set.csv', index_col = 0) # no need to have Y when do PSI analysis
# Import new data
new_data = pd.read_csv('original_data/loan_data_2015.csv', index_col = 0)

#dummies
parent_col = pd.read_csv('processed_data/dummy_parent_name.csv')['column'].to_list()
dummy_names = pd.read_csv('processed_data/pd_feature.csv')['feature'].to_list()

#score card
score_card = pd.read_csv('model_result/score_card.csv',index_col= 0)

/Users/warren/opt/anaconda3/envs/credit_risk/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (19,55) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# general quick view

In [4]:
new_data.head()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
68444620,73334399,35000,35000,35000.0,60 months,11.99,778.38,C,C1,Foreign Service Officer,10+ years,MORTGAGE,128000.0,Source Verified,Dec-15,Issued,n,https://www.lendingclub.com/browse/loanDetail....,NaN,home_improvement,Home improvement,200xx,DC,6.46,0.0,Feb-90,0.0,46.0,NaN,17.0,0.0,14277,27.4,46.0,w,35000.0,35000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,Jan-16,Jan-16,0.0,56.0,1,INDIVIDUAL,NaN,NaN,NaN,0.0,321.0,146867.0,1.0,11.0,0.0,0.0,28.0,35367.0,49.3,0.0,1.0,5020.0,40.1,52200.0,1.0,4.0,0.0
68547583,73437441,8650,8650,8650.0,36 months,5.32,260.50,A,A1,Associate Consultant,< 1 year,MORTGAGE,100000.0,Not Verified,Dec-15,Issued,n,https://www.lendingclub.com/browse/loanDetail....,NaN,credit_card,Credit card refinancing,462xx,IN,7.28,0.0,Jul-01,0.0,NaN,NaN,15.0,0.0,7158,26.7,24.0,w,8650.0,8650.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,Jan-16,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,0.0,165450.0,0.0,1.0,1.0,1.0,11.0,24041.0,88.8,0.0,3.0,3081.0,57.9,26800.0,1.0,0.0,5.0
67849662,72708407,4225,4225,4225.0,36 months,14.85,146.16,C,C5,mechanic,5 years,RENT,35000.0,Source Verified,Dec-15,Issued,n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,Debt consolidation,672xx,KS,15.22,2.0,Jul-11,0.0,18.0,NaN,6.0,0.0,1058,24.6,6.0,w,4225.0,4225.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,Jan-16,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,0.0,4888.0,0.0,1.0,0.0,0.0,47.0,3830.0,21.9,0.0,0.0,367.0,22.4,4300.0,0.0,0.0,0.0
68506885,73396712,10000,10000,10000.0,60 months,11.99,222.40,C,C1,CARDIOVASCULAR TECH,10+ years,RENT,42500.0,Not Verified,Dec-15,Issued,n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,Debt consolidation,460xx,IN,31.04,0.0,Dec-98,1.0,NaN,NaN,10.0,0.0,5812,40.9,23.0,w,10000.0,10000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,Jan-16,Jan-16,0.0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0.0,0.0,41166.0,1.0,3.0,0.0,2.0,13.0,35354.0,75.5,1.0,1.0,3118.0,67.4,14200.0,1.0,1.0,1.0
68341763,72928789,20000,20000,20000.0,60 months,10.78,432.66,B,B4,truck driver,10+ years,MORTGAGE,63000.0,Not Verified,Dec-15,Issued,n,https://www.lendingclub.com/browse/loanDetail....,NaN,home_improvement,NaN,605xx,IL,10.78,0.0,Aug-00,0.0,NaN,NaN,6.0,0.0,7869,56.2,18.0,w,20000.0,20000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,Jan-16,Dec-15,0.0,NaN,1,JOINT,71000.0,13.85,Not Verified,0.0,0.0,189699.0,0.0,1.0,0.0,4.0,19.0,10827.0,72.8,0.0,2.0,2081.0,64.7,14000.0,2.0,5.0,1.0


In [5]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421094 entries, 68444620 to 36271262
Data columns (total 73 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   member_id                    421094 non-null  int64  
 1   loan_amnt                    421094 non-null  int64  
 2   funded_amnt                  421094 non-null  int64  
 3   funded_amnt_inv              421094 non-null  float64
 4   term                         421094 non-null  object 
 5   int_rate                     421094 non-null  float64
 6   installment                  421094 non-null  float64
 7   grade                        421094 non-null  object 
 8   sub_grade                    421094 non-null  object 
 9   emp_title                    397220 non-null  object 
 10  emp_length                   397277 non-null  object 
 11  home_ownership               421094 non-null  object 
 12  annual_inc                   421094 non-null  flo

# Preprocessing

## Cleaning

### emp_length

In [6]:
col = 'emp_length'
Counter(new_data[col]).most_common()

[('10+ years', 141520),
 ('2 years', 37497),
 ('< 1 year', 34340),
 ('3 years', 33430),
 ('1 year', 27473),
 ('5 years', 24930),
 ('4 years', 24506),
 (nan, 23817),
 ('8 years', 21560),
 ('7 years', 18414),
 ('6 years', 16838),
 ('9 years', 16769)]

In [7]:
col = col+'_int'
new_data[col] = new_data['emp_length'].str.replace('\+ years', '')
new_data[col] = new_data[col].str.replace('< 1 year', str(0))
new_data[col] = new_data[col].str.replace(' years', '')
new_data[col] = new_data[col].str.replace(' year', '')
new_data[col] = new_data[col].str.replace('n/a', '0')
new_data[col] = np.where(new_data[col].isna(), '0', new_data[col])

new_data[col] = pd.to_numeric(new_data[col])
Counter(new_data[col]).most_common()

[(10, 141520),
 (0, 58157),
 (2, 37497),
 (3, 33430),
 (1, 27473),
 (5, 24930),
 (4, 24506),
 (8, 21560),
 (7, 18414),
 (6, 16838),
 (9, 16769)]

### earliest_cr_line

In [8]:
col ='earliest_cr_line'
count = Counter(new_data[col]).most_common()
len(count)
#count

668

In [9]:
# to date time
new_data[col+'_date'] = pd.to_datetime(new_data[col], format = '%b-%y')
new_data[col+'_date']

id
68444620   1990-02-01
68547583   2001-07-01
67849662   2011-07-01
68506885   1998-12-01
68341763   2000-08-01
              ...    
36371250   2004-09-01
36441262   1974-03-01
36271333   2003-09-01
36490806   2003-10-01
36271262   2001-12-01
Name: earliest_cr_line_date, Length: 421094, dtype: datetime64[ns]

In [10]:
# Assume we are now in December 2017, get monthes from now
new_data['mths_since_earliest_cr_line'] = round(
    pd.to_numeric(
        (pd.to_datetime('2018-12-01') - new_data[col+'_date'] ) / np.timedelta64(1, 'M')
    ))

In [11]:
new_data['mths_since_earliest_cr_line'].describe()

count    421094.000000
mean        239.354876
std          98.123989
min        -600.000000
25%         179.000000
50%         224.000000
75%         289.000000
max         599.000000
Name: mths_since_earliest_cr_line, dtype: float64

In [12]:
# there are some negative value, see what happen
new_data[[col, col+'_date']][new_data['mths_since_earliest_cr_line']<0]
# they are date in 2060 sth
# we here assume they are 1960 sth, and use the maximum months from now

,earliest_cr_line,earliest_cr_line_date
id,,
68406292,Jan-57,2057-01-01
68506307,Aug-64,2064-08-01
68466525,Jul-68,2068-07-01
67858278,Jan-65,2065-01-01
68595008,Sep-64,2064-09-01
...,...,...
38018510,Jun-68,2068-06-01
37801699,Jan-66,2066-01-01
37691363,Sep-68,2068-09-01


In [13]:
new_data['mths_since_earliest_cr_line'][
    new_data['mths_since_earliest_cr_line']<0] = new_data['mths_since_earliest_cr_line'].max()

<ipython-input-13-e87a85f57c9f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['mths_since_earliest_cr_line'][


In [14]:
new_data['mths_since_earliest_cr_line'].describe()

count    421094.000000
mean        242.471220
std          90.827636
min          73.000000
25%         180.000000
50%         224.000000
75%         290.000000
max         599.000000
Name: mths_since_earliest_cr_line, dtype: float64

### new_data

In [15]:
new_data['issue_d_date'] = pd.to_datetime(new_data['issue_d'], format = '%b-%y')
new_data['mths_since_issue_d'] = round(pd.to_numeric((pd.to_datetime('2018-12-01') - new_data['issue_d_date']) / np.timedelta64(1, 'M')))
new_data['mths_since_issue_d'].describe()

count    421094.000000
mean         41.044629
std           3.458162
min          36.000000
25%          38.000000
50%          41.000000
75%          44.000000
max          47.000000
Name: mths_since_issue_d, dtype: float64

### term

In [16]:
col = 'term'
Counter(new_data[col])

Counter({' 60 months': 137922, ' 36 months': 283172})

In [17]:
new_data[col+'_int'] = new_data[col].str.replace(' months', '')
new_data[col+'_int'] = new_data[col+'_int'].str.replace(' ','')
new_data[col+'_int'] = pd.to_numeric(new_data[col+'_int'])

print(
    new_data[col+'_int'].describe(),'\n\n',
    Counter(new_data[col+'_int'])
)


count    421094.000000
mean         43.860782
std          11.263533
min          36.000000
25%          36.000000
50%          36.000000
75%          60.000000
max          60.000000
Name: term_int, dtype: float64 

 Counter({36: 283172, 60: 137922})


## Missing

In [18]:
na_idx = new_data.isnull().sum()>0
list(new_data.columns.values[na_idx])

['emp_title',
 'emp_length',
 'desc',
 'title',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'revol_util',
 'last_pymnt_d',
 'next_pymnt_d',
 'last_credit_pull_d',
 'mths_since_last_major_derog',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'open_acc_6m',
 'open_il_6m',
 'open_il_12m',
 'open_il_24m',
 'mths_since_rcnt_il',
 'total_bal_il',
 'il_util',
 'open_rv_12m',
 'open_rv_24m',
 'max_bal_bc',
 'all_util',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m']

In [19]:
# 'Total revolving high credit/credit limit', so it makes sense that the missing values are equal to funded_amnt.
new_data['total_rev_hi_lim'].fillna(new_data['funded_amnt'], inplace=True)

In [20]:
new_data['acc_now_delinq'].fillna(0, inplace=True)
new_data['total_acc'].fillna(0, inplace=True)
new_data['pub_rec'].fillna(0, inplace=True)
new_data['open_acc'].fillna(0, inplace=True)
new_data['inq_last_6mths'].fillna(0, inplace=True)
new_data['delinq_2yrs'].fillna(0, inplace=True)
new_data['emp_length_int'].fillna(0, inplace=True)
new_data['mths_since_earliest_cr_line'].fillna(0, inplace=True)

In [21]:
new_data['annual_inc'].fillna(new_data['annual_inc'].mean(), inplace=True)

## create Y label

In [22]:
# Good/ Bad Definition
new_data[Y_lab] = np.where(new_data['loan_status'].isin(['Charged Off', 'Default',
                                                       'Does not meet the credit policy. Status:Charged Off',
                                                       'Late (31-120 days)']), 0, 1)

# Feature engineering

In [23]:
data = new_data.copy()

## discrete variable

In [24]:
col = 'home_ownership'
data[col] = data[col].replace(
    ['RENT', 'OTHER', 'NONE', 'ANY'],
    'RENT_OTHER_NONE_ANY')

In [25]:
col = 'addr_state'
group = [["ND", "NE", "IA", "NV" ,"FL", "HI", "AL"], 
         ["NM","VA"],
         ['OK','TN','MO','LA','MD','NC'],
         ['UT','KY','AZ','NJ'],
         ['AR','MI','PA','OH','MN'],
         ['RI','MA','DE','SD','IN'],
         ['GA','WA','OR'],
         ['WI','MT'],
         ['IL','CT'],
         ['KS','SC','CO','VT','AK','MS'],
         ['WV','NH','WY','DC','ME','ID']
        ]

for g in group:
    data[col] = data[col].replace(
    g,
    '_'.join(g))

Counter(data[col])

Counter({'WV_NH_WY_DC_ME_ID': 6463,
         'RI_MA_DE_SD_IN': 20702,
         'KS_SC_CO_VT_AK_MS': 22299,
         'IL_CT': 23191,
         'GA_WA_OR': 27971,
         'NM_VA': 14384,
         'AR_MI_PA_OH_MN': 51750,
         'OK_TN_MO_LA_MD_NC': 44763,
         'NY': 33844,
         'ND_NE_IA_NV_FL_HI_AL': 44293,
         'UT_KY_AZ_NJ': 31843,
         'TX': 34699,
         'CA': 58067,
         'WI_MT': 6825})

In [26]:
col = 'purpose'
group = [['educational','small_business','renewable_energy','moving'],
        ['other','wedding','medical','vacation','house'],
        ['major_purchase','car','home_improvement']]

group_abbr = [['educ','sm_b','ren_en','mov'],
        ['oth','med','vacation','house','wedd'],
        ['major_purch','car','home_impr']]


for i in range(0,3):
    data[col]= data[col].replace(
        group[i],
        '__'.join(group_abbr[i]))
    
Counter(data[col])    

Counter({'major_purch__car__home_impr': 36207,
         'credit_card': 102025,
         'debt_consolidation': 250020,
         'oth__med__vacation__house__wedd': 26833,
         'educ__sm_b__ren_en__mov': 6009})

## Continuous Variable

In [27]:
def  continuous_cutter(data, cuts, col, level = 1, dec = 0):
    #cuts should be a list (i) of lists or num (j); for each j, if j = list there are 2 num represent lr,ur
    #eg.cuts=[0,1,[2,5],[6,7],[8,10],10]

    if level==1000: level_cat ='K'
    else: level_cat=''

    temp_list = ['']*data.shape[0]
    #train_temp_list = ['']*X_train.shape[0]
    for i in range(len(cuts)):

        # min
        if i == 0:
            #print('beg')
            if type(cuts[0]) !=list:
                lab = '<=' + str(cuts[0]) + level_cat
                temp_list = np.where(data[col]<=cuts[0]*level, lab, temp_list)
                #train_temp_list = np.where(X_train[col]<=cuts[0]*level, lab, train_temp_list)
            else: 
                lr = cuts[i][0]
                ur = cuts[i][1]
                lab = '-'.join([str(round(lr,dec))+ level_cat , str(round(ur,dec))+ level_cat])
                temp_list = np.where(data[col].between(lr*level,ur*level), lab, temp_list)
                #train_temp_list = np.where(X_train[col].between(lr*level,ur*level), lab, train_temp_list)

                lab = '<' + str(round(lr,dec)) + level_cat
                temp_list = np.where(data[col]<lr*level, lab, temp_list)
                #train_temp_list = np.where(X_train[col]<lr*level, lab, train_temp_list)

        #max
        elif i==(len(cuts)-1):
            #print('max')
            if type(cuts[i]) !=list:
                lab = '>=' + str(round(cuts[-1],dec)) + level_cat
                temp_list = np.where(data[col]>=cuts[-1]*level, lab, temp_list)
                #train_temp_list = np.where(X_train[col]>=cuts[-1]*level, lab, train_temp_list)
            else: 
                lr = cuts[i][0]
                ur = cuts[i][1]
                lab = '-'.join([str(round(lr,dec))+ level_cat , str(round(ur,dec))+ level_cat])
                temp_list = np.where(data[col].between(lr*level,ur*level), lab, temp_list)
                #train_temp_list = np.where(X_train[col].between(lr*level,ur*level), lab, train_temp_list)

                lab = '>' + str(round(ur,dec)) + level_cat
                temp_list = np.where(data[col]>lr*level, lab, temp_list)
                #train_temp_list = np.where(X_train[col]>lr*level, lab, train_temp_list)

        #between
        else:
            #print('bet')
            if type(cuts[i]) !=list:
                lab = '=' + str(cuts[i]) + level_cat
                temp_list = np.where(data[col]==cuts[i]*level, lab, temp_list)
                #train_temp_list = np.where(X_train[col]==cuts[i]*level, lab, train_temp_list)
            else: 
                lr = cuts[i][0]
                ur = cuts[i][1]
                lab = '-'.join([str(round(lr,dec))+ level_cat , str(round(ur,dec))+ level_cat])
                temp_list = np.where(data[col].between(lr*level,ur*level), lab, temp_list)
                #train_temp_list = np.where(X_train[col].between(lr*level,ur*level), lab, train_temp_list)

    # missing value
    temp_list = np.where(temp_list=='', 'Missing', temp_list)
    #train_temp_list = np.where(train_temp_list=='', 'Missing', train_temp_list)
    
    return temp_list#, train_temp_list

In [28]:
col = 'term_int'
data[col] = data[col].astype(str)

In [29]:
col = 'emp_length_int'
cuts=[0,
      1,
     [2,5],
     [6,7],
     [8,10],
     10]

temp_list = continuous_cutter(data, cuts = cuts, col= col, level = 1, dec = 0 )

data[col]= temp_list
# check
Counter(temp_list)

Counter({'>=10': 141520,
         '<=0': 58157,
         '2-5': 120363,
         '8-10': 38329,
         '=1': 27473,
         '6-7': 35252})

In [30]:
col = 'mths_since_issue_d'
cuts=[38,
      [38.001,40],
     [40.001,42],
     [42.001,49],
     [49.001,53],
     [49.001,53],
     [53.001,65],
     [65.001,85],
     85.001]

temp_list = continuous_cutter(data, cuts = cuts, col= col, level = 1, dec = 0 )
data[col]= temp_list

# check
Counter(temp_list)

Counter({'<=38': 130503,
         '38.0-40': 64527,
         '40.0-42': 74447,
         '42.0-49': 151617})

In [31]:
col = 'int_rate'
cuts=[[9.548, 12.025],
     [12.026,15.74],
     [15.741,20.281],
     20.282]

temp_list = continuous_cutter(data, cuts = cuts, col= col, level = 1, dec = 3 )
data[col]= temp_list
# check
Counter(temp_list)

Counter({'9.548-12.025': 76182,
         '<9.548': 117624,
         '12.026-15.74': 134429,
         '>=20.282': 18469,
         '15.741-20.281': 74390})

In [32]:
col = 'funded_amnt'
ij = np.round(np.linspace(start=500, stop=35000, num=50),0)
cuts = [[i,j] for i, j in zip(ij+ np.log(1.0001), ij[1:])]

temp_list = continuous_cutter(data, cuts = cuts, col= col, level = 1, dec = 0 )

# check
data[col]= temp_list
Counter(temp_list)

Counter({'>35000.0': 20225,
         '8245.0-8949.0': 6188,
         '4020.0-4724.0': 4043,
         '9653.0-10357.0': 32269,
         '19510.0-20214.0': 26362,
         '14582.0-15286.0': 26647,
         '24439.0-25143.0': 14253,
         '3316.0-4020.0': 8980,
         '11765.0-12469.0': 27790,
         '23735.0-24439.0': 13803,
         '10357.0-11061.0': 9480,
         '17398.0-18102.0': 13735,
         '22327.0-23031.0': 3869,
         '8949.0-9653.0': 11781,
         '7541.0-8245.0': 16148,
         '27959.0-28663.0': 9008,
         '1908.0-2612.0': 5751,
         '13878.0-14582.0': 12148,
         '15990.0-16694.0': 14210,
         '29367.0-30071.0': 10532,
         '28663.0-29367.0': 1352,
         '18806.0-19510.0': 5102,
         '4724.0-5429.0': 18214,
         '15286.0-15990.0': 3588,
         '20918.0-21622.0': 8731,
         '6837.0-7541.0': 13996,
         '27255.0-27959.0': 1285,
         '23031.0-23735.0': 1596,
         '11061.0-11765.0': 6032,
         '5429.0-6133.0

In [33]:
col = 'mths_since_earliest_cr_line'
cuts = [[140.001, 165],
        [165.001, 248],
        [248.001, 271],
        [271.001,353]
        ]


temp_list = continuous_cutter(data, cuts = cuts, col= col, level = 1, dec = 0 )
data[col]= temp_list
# check
Counter(temp_list)

Counter({'>353': 129730,
         '165.0-248': 182443,
         '<140.0': 37493,
         '140.0-165': 37569,
         '248.0-271': 33859})

In [34]:
col = 'delinq_2yrs'
cuts = [0,
        [1,3],
        4]

temp_list = continuous_cutter(data, cuts = cuts, col= col, level = 1, dec = 0 )
data[col]= temp_list
# check
Counter(temp_list)

Counter({'<=0': 334007, '1-3': 80534, '>=4': 6553})

In [35]:
col = 'inq_last_6mths'
cuts = [0,
        [1,2],
        [3,6],
        7
       ]

temp_list = continuous_cutter(data, cuts = cuts, col= col, level = 1, dec = 0 )
data[col]= temp_list
# check
Counter(temp_list)

Counter({'<=0': 256407, '1-2': 147638, '3-6': 17049})

In [36]:
col = 'open_acc'
cuts = [0,
        [1,3],
        [4,12],
        [13,17],
        [18,22],
        [23,25],
        [26,30],
        31
       ]

temp_list = continuous_cutter(data, cuts = cuts, col= col, level = 1, dec = 0 )
data[col]= temp_list
# check
Counter(temp_list)


Counter({'13-17': 99835,
         '4-12': 254430,
         '26-30': 7173,
         '18-22': 40426,
         '>=31': 3675,
         '1-3': 5195,
         '23-25': 10360})

In [37]:
col = 'pub_rec'
cuts=[[0,2],
     [3,4],
     5]

temp_list = continuous_cutter(data, cuts = cuts, col= col, level = 1, dec = 0 )
data[col]= temp_list
# check
Counter(temp_list)


Counter({'0-2': 416124, '3-4': 3920, '>=5': 1050})

In [38]:
col = 'total_acc'
cuts=[27,
     [28,51],
     52]

temp_list = continuous_cutter(data, cuts = cuts, col= col, level = 1, dec = 0 )
data[col]= temp_list
# check
Counter(temp_list)

Counter({'28-51': 145715, '<=27': 261827, '>=52': 13552})

In [39]:
col = 'acc_now_delinq'
cuts=[0,
     1]

temp_list = continuous_cutter(data, cuts = cuts, col= col, level = 1, dec = 0 )
data[col]= temp_list
# check
Counter(temp_list)


Counter({'<=0': 418712, '>=1': 2382})

In [40]:
col = 'total_rev_hi_lim'
cuts=[5,
     [5.0001,10],
     [10.0001,20],
     [20.00001,30],
     [30.00001,40],
     [40.00001,55],
     [55.00001,95],
     95]

temp_list = continuous_cutter(data, cuts = cuts, col= col, level = 1000, dec = 0 )
data[col]= temp_list
# check
Counter(temp_list)


Counter({'40.0K-55K': 49650,
         '20.0K-30K': 85112,
         '<=5K': 13805,
         '10.0K-20K': 110856,
         '30.0K-40K': 55706,
         '5.0K-10K': 43351,
         '>=95K': 17519,
         '55.0K-95K': 45095})

In [41]:
col = 'installment'
ij = np.round(np.linspace(start=14.356, stop=1409.99, num=50),0)
cuts = [[i,j] for i, j in zip(ij+ np.log(1.0001), ij[1:])]

temp_list = continuous_cutter(data, cuts = cuts, col= col, level = 1, dec = 0 )
data[col]= temp_list
# check
Counter(temp_list)


Counter({'755.0-783.0': 7753,
         '242.0-271.0': 25899,
         '128.0-157.0': 10951,
         '214.0-242.0': 17732,
         '413.0-442.0': 13185,
         '470.0-499.0': 18965,
         '100.0-128.0': 8595,
         '385.0-413.0': 18557,
         '442.0-470.0': 17361,
         '499.0-527.0': 15090,
         '271.0-299.0': 21054,
         '584.0-612.0': 9467,
         '299.0-328.0': 29735,
         '812.0-840.0': 4183,
         '840.0-869.0': 5301,
         '71.0-100.0': 6425,
         '527.0-556.0': 13245,
         '356.0-385.0': 22817,
         '556.0-584.0': 11121,
         '328.0-356.0': 24292,
         '1040.0-1068.0': 1000,
         '612.0-641.0': 11738,
         '157.0-185.0': 18194,
         '669.0-698.0': 9315,
         '641.0-669.0': 10003,
         '726.0-755.0': 6601,
         '783.0-812.0': 6437,
         '954.0-983.0': 2130,
         '185.0-214.0': 18145,
         '698.0-726.0': 5285,
         '43.0-71.0': 4566,
         '1068.0-1097.0': 2110,
         '926.0-954.0

In [42]:
col = 'annual_inc'
cuts=[20,
     [20.00001,30],
     [30.00001,40],
     [20.00001,50],
     [50.00001,60],
     [60.00001,70],
     [70.00001,80],
     [80.00001,90],
     [90.00001,100],
     [100.00001,120],
     [120.00001,140],
     140]

temp_list = continuous_cutter(data, cuts = cuts, col= col, level = 1000, dec = 0 )

# check
Counter(temp_list)
data[col]= temp_list

In [43]:
col = 'mths_since_last_delinq'
cuts=[[0,3],
     [4,30],
     [31,56],
     57]

temp_list = continuous_cutter(data, cuts = cuts, col= col, level = 1, dec = 0 )
data[col]= temp_list
# check
Counter(temp_list)


Counter({'31-56': 66139,
         'Missing': 203961,
         '4-30': 100930,
         '>=57': 42803,
         '0-3': 7261})

In [44]:
col = 'dti'
cuts=[1.4,
      [1.41,3.5],
     [3.51,7.7],
     [7.71,10.5],
     [10.51,16.1],
     [16.11,20.3],
     [20.31,21.7],
     [21.71,22.4],
     [22.41,35],
     35.01]

temp_list = continuous_cutter(data, cuts = cuts, col= col, level = 1, dec = 1 )
data[col]= temp_list
# check
Counter(temp_list)


Counter({'3.5-7.7': 30805,
         '10.5-16.1': 92579,
         '22.4-35': 130422,
         '7.7-10.5': 34187,
         '16.1-20.3': 74060,
         '20.3-21.7': 22837,
         '21.7-22.4': 10849,
         '>=35.0': 16923,
         '1.4-3.5': 6609,
         '<=1.4': 1823})

In [45]:
col = 'mths_since_last_record'
cuts=[[0,2],
      [3,20],
     [21,31],
     [32,80],
     [81,86],
     86.01]

temp_list = continuous_cutter(data, cuts = cuts, col= col, level = 1, dec = 0 )
data[col]= temp_list
# check
Counter(temp_list)

Counter({'Missing': 346679,
         '32-80': 46908,
         '21-31': 3723,
         '0-2': 87,
         '3-20': 3099,
         '>=86.0': 15212,
         '81-86': 5386})

# prepare data

## create dummy (new data)

### new data

In [46]:
# get dummies
data_dummies = pd.concat(
    [pd.get_dummies(data[c], prefix = c, prefix_sep = col_sep) for c in parent_col],
    axis = 1)
    
# concat data
new_data = pd.concat([data, data_dummies], axis = 1)

In [47]:
# check what dummis that appear in old data are missing in the new data
idx = [d not in new_data.columns.values.tolist() for d in dummy_names]
miss_col = np.array(dummy_names)[idx].tolist()
miss_col

['mths_since_issue_d:49.0-53',
 'mths_since_issue_d:53.0-65',
 'mths_since_issue_d:65.0-85',
 'mths_since_issue_d:>=85.0',
 'inq_last_6mths:>=7',
 'open_acc:<=0']

In [48]:
# insert the missing dummies
for c in miss_col:
    new_data[c] = 0

In [49]:
new_data_backup = new_data.copy() # later save use, ie. if we want to fit model use this data, no need to clean again

## select and insert columns

In [50]:
# intercept
old_data.insert(0, 'Intercept', 1)
new_data.insert(0, 'Intercept', 1)


In [51]:
score_card

,group,value,feature,credit_score,coefficient,p_value
0,Intercept,NaN,Intercept,309.0,-1.726363,NaN
3,addr_state,AR_MI_PA_OH_MN,addr_state:AR_MI_PA_OH_MN,12.0,0.155171,4.474558e-12
4,addr_state,CA,addr_state:CA,7.0,0.095189,6.767859e-06
5,addr_state,GA_WA_OR,addr_state:GA_WA_OR,14.0,0.190961,6.021733e-13
6,addr_state,IL_CT,addr_state:IL_CT,22.0,0.291465,1.910721e-23
7,addr_state,KS_SC_CO_VT_AK_MS,addr_state:KS_SC_CO_VT_AK_MS,26.0,0.343183,7.813634e-29
85,addr_state,ND_NE_IA_NV_FL_HI_AL,addr_state:ND_NE_IA_NV_FL_HI_AL,0.0,0.000000,NaN
8,addr_state,NM_VA,addr_state:NM_VA,7.0,0.093430,3.773944e-03
9,addr_state,NY,addr_state:NY,5.0,0.061618,1.044822e-02
10,addr_state,OK_TN_MO_LA_MD_NC,addr_state:OK_TN_MO_LA_MD_NC,7.0,0.091401,7.751581e-05


In [52]:
# select only features that appears in score card
idx = score_card.feature.tolist()
old_data = old_data[idx]
new_data = new_data[idx]

## score card reshape

In [53]:
credit_score = score_card.credit_score.values
credit_score = credit_score.reshape((len(credit_score),1))
credit_score.shape

(101, 1)

## calculate credit score

In [54]:
old_data['score'] = old_data.dot(credit_score)
new_data['score'] = new_data.dot(credit_score)

## create score categories in data

In [55]:
ij = np.linspace(300,850, 12)
categories = [[int(i),int(j)] for i, j in zip(ij + 1, ij[1:]) ]
categories[0][0] = 300

for i in categories:
    cat = 'score:' + '-'.join([str(i[0]),str(i[1])])
    old_data[cat] = np.where(old_data['score'].between(i[0],i[1]),1,0)
    new_data[cat] = np.where(new_data['score'].between(i[0],i[1]),1,0)    
                    

In [56]:
old_data.head()

,Intercept,addr_state:AR_MI_PA_OH_MN,addr_state:CA,addr_state:GA_WA_OR,addr_state:IL_CT,addr_state:KS_SC_CO_VT_AK_MS,addr_state:ND_NE_IA_NV_FL_HI_AL,addr_state:NM_VA,addr_state:NY,addr_state:OK_TN_MO_LA_MD_NC,addr_state:RI_MA_DE_SD_IN,addr_state:TX,addr_state:UT_KY_AZ_NJ,addr_state:WI_MT,addr_state:WV_NH_WY_DC_ME_ID,annual_inc:100.0K-120K,annual_inc:120.0K-140K,annual_inc:20.0K-50K,annual_inc:50.0K-60K,annual_inc:60.0K-70K,annual_inc:70.0K-80K,annual_inc:80.0K-90K,annual_inc:90.0K-100K,annual_inc:<=20K,annual_inc:>=140K,dti:1.4-3.5,dti:10.5-16.1,dti:16.1-20.3,dti:20.3-21.7,dti:21.7-22.4,dti:22.4-35,dti:3.5-7.7,dti:7.7-10.5,dti:<=1.4,dti:>=35.0,emp_length_int:2-5,emp_length_int:6-7,emp_length_int:8-10,emp_length_int:<=0,emp_length_int:=1,emp_length_int:>=10,home_ownership:MORTGAGE,home_ownership:OWN,home_ownership:RENT_OTHER_NONE_ANY,initial_list_status:f,initial_list_status:w,inq_last_6mths:1-2,inq_last_6mths:3-6,inq_last_6mths:<=0,inq_last_6mths:>=7,int_rate:12.026-15.74,int_rate:15.741-20.281,int_rate:9.548-12.025,int_rate:<9.548,int_rate:>=20.282,mths_since_earliest_cr_line:140.0-165,mths_since_earliest_cr_line:165.0-248,mths_since_earliest_cr_line:248.0-271,mths_since_earliest_cr_line:<140.0,mths_since_earliest_cr_line:>353,mths_since_issue_d:38.0-40,mths_since_issue_d:40.0-42,mths_since_issue_d:42.0-49,mths_since_issue_d:49.0-53,mths_since_issue_d:53.0-65,mths_since_issue_d:65.0-85,mths_since_issue_d:<=38,mths_since_issue_d:>=85.0,mths_since_last_delinq:0-3,mths_since_last_delinq:31-56,mths_since_last_delinq:4-30,mths_since_last_delinq:>=57,mths_since_last_delinq:Missing,mths_since_last_record:0-2,mths_since_last_record:21-31,mths_since_last_record:3-20,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>=86.0,mths_since_last_record:Missing,purpose:credit_card,purpose:debt_consolidation,purpose:educ__sm_b__ren_en__mov,purpose:major_purch__car__home_impr,purpose:oth__med__vacation__house__wedd,term_int:36,term_int:60,total_acc:28-51,total_acc:<=27,total_acc:>=52,total_rev_hi_lim:10.0K-20K,total_rev_hi_lim:20.0K-30K,total_rev_hi_lim:30.0K-40K,total_rev_hi_lim:40.0K-55K,total_rev_hi_lim:5.0K-10K,total_rev_hi_lim:55.0K-95K,total_rev_hi_lim:<=5K,total_rev_hi_lim:>=95K,verification_status:Not Verified,verification_status:Source Verified,verification_status:Verified,score,score:300-350,score:351-400,score:401-450,score:451-500,score:501-550,score:551-600,score:601-650,score:651-700,score:701-750,score:751-800,score:801-850
279464,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,709.0,0,0,0,0,0,0,0,0,1,0,0
341098,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,618.0,0,0,0,0,0,0,1,0,0,0,0
435552,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,614.0,0,0,0,0,0,0,1,0,0,0,0
34346,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,624.0,0,0,0,0,0,0,1,0,0,0,0
54308,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,592.0,0,0,0,0,0,1,0,0,0,0,0


In [57]:
new_data.head()

,Intercept,addr_state:AR_MI_PA_OH_MN,addr_state:CA,addr_state:GA_WA_OR,addr_state:IL_CT,addr_state:KS_SC_CO_VT_AK_MS,addr_state:ND_NE_IA_NV_FL_HI_AL,addr_state:NM_VA,addr_state:NY,addr_state:OK_TN_MO_LA_MD_NC,addr_state:RI_MA_DE_SD_IN,addr_state:TX,addr_state:UT_KY_AZ_NJ,addr_state:WI_MT,addr_state:WV_NH_WY_DC_ME_ID,annual_inc:100.0K-120K,annual_inc:120.0K-140K,annual_inc:20.0K-50K,annual_inc:50.0K-60K,annual_inc:60.0K-70K,annual_inc:70.0K-80K,annual_inc:80.0K-90K,annual_inc:90.0K-100K,annual_inc:<=20K,annual_inc:>=140K,dti:1.4-3.5,dti:10.5-16.1,dti:16.1-20.3,dti:20.3-21.7,dti:21.7-22.4,dti:22.4-35,dti:3.5-7.7,dti:7.7-10.5,dti:<=1.4,dti:>=35.0,emp_length_int:2-5,emp_length_int:6-7,emp_length_int:8-10,emp_length_int:<=0,emp_length_int:=1,emp_length_int:>=10,home_ownership:MORTGAGE,home_ownership:OWN,home_ownership:RENT_OTHER_NONE_ANY,initial_list_status:f,initial_list_status:w,inq_last_6mths:1-2,inq_last_6mths:3-6,inq_last_6mths:<=0,inq_last_6mths:>=7,int_rate:12.026-15.74,int_rate:15.741-20.281,int_rate:9.548-12.025,int_rate:<9.548,int_rate:>=20.282,mths_since_earliest_cr_line:140.0-165,mths_since_earliest_cr_line:165.0-248,mths_since_earliest_cr_line:248.0-271,mths_since_earliest_cr_line:<140.0,mths_since_earliest_cr_line:>353,mths_since_issue_d:38.0-40,mths_since_issue_d:40.0-42,mths_since_issue_d:42.0-49,mths_since_issue_d:49.0-53,mths_since_issue_d:53.0-65,mths_since_issue_d:65.0-85,mths_since_issue_d:<=38,mths_since_issue_d:>=85.0,mths_since_last_delinq:0-3,mths_since_last_delinq:31-56,mths_since_last_delinq:4-30,mths_since_last_delinq:>=57,mths_since_last_delinq:Missing,mths_since_last_record:0-2,mths_since_last_record:21-31,mths_since_last_record:3-20,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>=86.0,mths_since_last_record:Missing,purpose:credit_card,purpose:debt_consolidation,purpose:educ__sm_b__ren_en__mov,purpose:major_purch__car__home_impr,purpose:oth__med__vacation__house__wedd,term_int:36,term_int:60,total_acc:28-51,total_acc:<=27,total_acc:>=52,total_rev_hi_lim:10.0K-20K,total_rev_hi_lim:20.0K-30K,total_rev_hi_lim:30.0K-40K,total_rev_hi_lim:40.0K-55K,total_rev_hi_lim:5.0K-10K,total_rev_hi_lim:55.0K-95K,total_rev_hi_lim:<=5K,total_rev_hi_lim:>=95K,verification_status:Not Verified,verification_status:Source Verified,verification_status:Verified,score,score:300-350,score:351-400,score:401-450,score:451-500,score:501-550,score:551-600,score:601-650,score:651-700,score:701-750,score:751-800,score:801-850
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
68444620,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,1,0,758.0,0,0,0,0,0,0,0,0,0,1,0
68547583,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,778.0,0,0,0,0,0,0,0,0,0,1,0
67849662,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,646.0,0,0,0,0,0,0,1,0,0,0,0
68506885,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,659.0,0,0,0,0,0,0,0,1,0,0,0
68341763,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,730.0,0,0,0,0,0,0,0,0,1,0,0


# PSI: Population Stability Index: Calculation and Interpretation

## prepare

In [58]:
idx = [c not in ['Intercept', 'score'] for c in old_data.columns.to_list() ]# drop intercept and score
old_data_percent = old_data.loc[:,idx].mean() 
new_data_percent = new_data.loc[:,idx].mean() 

cal_psi = pd.concat([old_data_percent, new_data_percent] ,axis= 1, )
cal_psi.columns = ['percentage_old', 'percentage_new']
cal_psi = cal_psi.reset_index()
cal_psi['feature_group'] = cal_psi['index'].str.split(':').str[0]

## PSI calculation

In [59]:
## PSI calculate
old = cal_psi['percentage_old']
new = cal_psi['percentage_new']
cal_psi['contribution'] = np.where((old == 0) | (new == 0), 0, (new - old) * np.log(new / old))

cal_psi.sort_values('contribution', ascending= False)

/Users/warren/opt/anaconda3/envs/credit_risk/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,index,percentage_old,percentage_new,feature_group,contribution
44,initial_list_status:w,0.350097,0.634659,initial_list_status,0.169279
43,initial_list_status:f,0.649903,0.365341,initial_list_status,0.163905
65,mths_since_issue_d:<=38,0.158787,0.309914,mths_since_issue_d,0.101064
104,score:501-550,0.087728,0.020941,score,0.095676
59,mths_since_issue_d:38.0-40,0.063100,0.153237,mths_since_issue_d,0.079975
108,score:701-750,0.102775,0.208063,score,0.074259
105,score:551-600,0.256522,0.145454,score,0.063015
33,dti:>=35.0,0.007678,0.040188,dti,0.053813
60,mths_since_issue_d:40.0-42,0.099966,0.176794,mths_since_issue_d,0.043805
52,int_rate:<9.548,0.179893,0.279330,int_rate,0.043755


## PSI
measure how much alike two samples are

PSI = (% of records based on scoring variable in Sample (A) - % of records based on scoring variable in Sample (B)) * In(A/ B)

In [60]:
PSI = cal_psi.groupby('feature_group')['contribution'].sum()
PSI


feature_group
addr_state                     0.003808
annual_inc                     0.005116
dti                            0.078814
emp_length_int                 0.011313
home_ownership                 0.004119
initial_list_status            0.333184
inq_last_6mths                 0.046666
int_rate                       0.079557
mths_since_earliest_cr_line    0.002929
mths_since_issue_d             0.267468
mths_since_last_delinq         0.011684
mths_since_last_record         0.055751
purpose                        0.008176
score                          0.306092
term_int                       0.012958
total_acc                      0.001132
total_rev_hi_lim               0.064680
verification_status            0.048934
Name: contribution, dtype: float64